In [ ]:
import torch
import Airfoil_DDPM_tools
import os
import pandas as pd
import numpy as np

In [ ]:
data_location=os.path.join("data","ACcontext_combined_m2.csv")
data = pd.read_csv(data_location)
data.fillna(0, inplace=True)

In [ ]:
print(data.shape)

In [ ]:
CST = data.iloc[:, :20]
ACC = data.iloc[:, 20:]

In [ ]:
print(CST.shape)

In [ ]:
CST = CST.to_numpy()

In [ ]:
arrays = [np.random.beta(a=0.5, b=0.5, size=(200, 2)) for _ in range(CST.shape[0])]
for i,cst in enumerate(CST):
    arrays[i] = Airfoil_DDPM_tools.CST2pointcloud(cst, 100)

point_cloud = np.stack(arrays, axis=0)  # 沿第0轴堆叠

In [ ]:
print(len(arrays),len(arrays[0]),len(arrays[0][0]))

In [ ]:
print(point_cloud.shape)

In [ ]:
'''
随机打乱，这里就不采用了，到训练的时候再打乱就好
'''
ACC_Numpy = ACC.to_numpy()
shuffle_idx = torch.randperm(CST.shape[0])
shuffled_pointcloud = point_cloud[shuffle_idx]
shuffled_ACC = (ACC.to_numpy())[shuffle_idx]

In [ ]:
print(shuffled_pointcloud.shape, shuffled_ACC.shape)

In [ ]:
# 保存到文件
save_path = os.path.join("../data","train_data_pointcloud.npz")
np.savez(save_path, 
         pointcloud=point_cloud, 
         ACC=ACC_Numpy)

In [ ]:
loaded_data = np.load(save_path)
loaded_pointcloud = loaded_data['pointcloud']  # 形状 [B, N, 3]
loaded_ACC = loaded_data['ACC']                # 形状 [B, 6]